# Horizons

REBOUND can add particles to simulations by obtaining ephemerides from NASA's powerful HORIZONS database.  HORIZONS supports many different options, and we will certainly not try to cover everything here.  This is meant to serve as an introduction to the basics, beyond what's in [Churyumov-Gerasimenko.ipynb](Churyumov-Gerasimenko.ipynb).  If you catch any errors, or would either like to expand on this documentation or improve REBOUND's HORIZONS interface (`rebound/horizons.py`), please do fork the repository and send us a pull request.

**Adding particles**

When we add particles by passing a string, REBOUND queries the HORIZONS database and takes the first dataset HORIZONS offers.  For the Sun, moons, and small bodies, this will typically return the body itself.  For planets, it will return the barycenter of the system (for moonless planets like Venus it will say barycenter but there is no distinction).  In all cases, REBOUND will print out the name of the HORIZONS entry it's using.

You can also add bodies using their integer NAIF IDs:  [NAIF IDs](http://naif.jpl.nasa.gov/pub/naif/toolkit_docs/MATLAB/req/naif_ids.html).  Note that because of the number of small bodies (asteroids etc.) we have discovered, this convention only works for large objetcts.  For small bodies, instead use "NAME=name" (see the SMALL BODIES section in the [HORIZONS Documentation](http://ssd.jpl.nasa.gov/?horizons_doc)).

To speed up our automatic builds/tests, we have changed the code sections of this tutorial to text, but feel free to test them out!

import rebound
rebound.add("Sun")
rebound.add("Venus")
rebound.add("399")
rebound.add("Europa")
rebound.add("NAME=Ida")
rebound.status()

Currently, HORIZONS does not have any mass information for solar system bodies.  `rebound/horizons.py` has a hard-coded list provided by Jon Giorgini (10 May 2015) that includes the planets, their barycenters (total mass of planet plus moons), and the largest moons.  If REBOUND doesn't find the corresponding mass for an object from this list (like for the asteroid Ida above), it will print a warning message.  If you need the body's mass for your simulation, you can set it manually, e.g. (see [Units.ipynb](Units.ipynb) for an overview of using different units):

In [2]:
rebound.particles[4] = 2.1e-14 # mass of Ida in Solar masses

**Time**

By default, REBOUND queries HORIZONS for objects' current positions.  Specifically, it caches the current time the first time you call `rebound.add`, and gets the corresponding ephemeris.  All subsequent calls to `rebound.add` will then use that initial cached time to make sure you get a synchronized set of ephemerides.

You can also explicitly pass REBOUND the time at which you would like the particles ephemerides:  

rebound.reset()
date = "2005-06-30 15:24"
rebound.add("Venus")
rebound.add("Venus", date=date)
rebound.status()

We see that the two Venus positions are different.  The first call cached the current time, but since the second call specified a date, it overrode the default.  Any time you pass a date, it will overwrite the default cached time, so: 

rebound.reset()
date = "2005-06-30 15:24"
rebound.add("Venus", date=date)
rebound.add("Earth")
rebound.add("Mercury")
rebound.status()

would set up a simulation with Mercury, Venus and Earth, all synchronized to 2005-06-30 15:24.  All dates should be passed in the format Year-Month-Day Hour:Minute.  

REBOUND takes these absolute times to the nearest minute, since at the level of seconds you have to worry about exactly what time system you're using, and small additional perturbations probably start to matter.  For reference HORIZONS interprets all times for ephemerides as [Coordinate (or Barycentric Dynamical) Time](https://en.wikipedia.org/wiki/Barycentric_Dynamical_Time).

**Reference Frame**

REBOUND queries for particles' positions and velocities relative to the Sun:

rebound.reset()
rebound.add("Sun")
rebound.status()

The reference plane is the ecliptic (Earth's orbital plane) of J2000 (Jan. 1st 2000 12:00 GMT), with the x axis along the ascending node of the ecliptic and the Earth's mean equator (also at J2000).  